# QA Labor law training file

Connect drive

In [18]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


## Preprocessing data functions

* Lib for preprocessing data

In [47]:
import json
import re

1. Load data

In [48]:
def load_all_data(input_file: str) -> dict:
    """
    Loads the data from the given input file.

    Args:
        input_file (str): The path to the input file containing the data.

    Returns:
        dict: The loaded data as a dictionary
    """
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

2. Clean text

In [49]:
def clean_text(text):
    """
    Cleans the given text by removing newlines, numeric annotations, special characters, and converting it to lowercase.

    Args:
        text (str): The text to be cleaned.

    Returns:
        str: The cleaned text.
    """
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\[\d+\]', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    return text

3. Preprocess data

In [50]:
def preprocess_data(data):
    """
    Preprocesses the given data and extracts contexts, questions, and answers, context_raws, question_raws.

    Args:
      data (dict): The input data containing articles, paragraphs, and questions.

    Returns:
      tuple: A tuple containing three lists - contexts, questions, and answers.
        - contexts (list): A list of preprocessed contexts.
        - questions (list): A list of preprocessed questions.
        - answers (list): A list of dictionaries, each containing the preprocessed answer text and its start position.
        - context_raws (list): A list of raw contexts.
        - question_raws (list): A list of raw questions.
    """
    contexts = []
    context_raws = []
    questions = []
    answers = []
    question_raws = []
    for dataSquad in data:
        for article in dataSquad['data']:
            for paragraph in article['paragraphs']:
                context = clean_text(paragraph['context'])
                for qa in paragraph['qas']:
                    question = clean_text(qa['question'])
                    # Do ngữ liệu nhiều chỗ define thiếu is_impossible nên sẽ mặc định là False
                    is_impossible = qa.get('is_impossible', False)
                    if not is_impossible:
                        for answer in qa['answers']:
                            answer_text = clean_text(answer['text'])
                            answer_start = answer['answer_start']
                            contexts.append(context)
                            context_raws.append(paragraph['context'])
                            questions.append(question)
                            question_raws.append(qa['question'])
                            answers.append({
                                'text': answer_text,
                                'start': answer_start
                            })
    return contexts, questions, answers, context_raws, question_raws

4. Tokenize input

In [51]:
def tokenize_texts(texts):
    """
    Tokenizes a list of texts using the word_tokenize function.

    Args:
      texts (list): A list of texts to be tokenized.

    Returns:
      list: A list of tokenized texts.
    """
    return [text.split() for text in texts]

* Running preprocessing data

In [55]:
data_path = './drive/MyDrive/models/qa_train.json'
squad_data = load_all_data(data_path)
contexts, questions, answers, context_raws, question_raws = preprocess_data(squad_data)
tokenized_contexts = tokenize_texts(contexts)
tokenized_questions = tokenize_texts(questions)

In [53]:
# Testing data
# tokenized_questions[:100]

## Word2vec model

1. Install lib for model word2vec

In [14]:
!pip install gensim

2. Lib import using word2vec

In [12]:
import gensim
import numpy as np
from gensim.utils import simple_preprocess

3. Load model



In [22]:
word2vec_questions = gensim.models.Word2Vec.load("./drive/MyDrive/models/word2vec_question.model")

4. Function model word2vec

In [23]:
def get_avg_word2vec_vector(user_question, model_word2vec):
    words = user_question
    word_vectors = [model_word2vec.wv[word] for word in words if word in model_word2vec.wv]
    if not word_vectors:
        return np.zeros(model_word2vec.vector_size)
    return np.mean(word_vectors, axis=0)

In [24]:
def get_word2vec_scores(user_question, datas, model_word2vec):
    query_vector = get_avg_word2vec_vector(user_question, model_word2vec)
    scores = []
    for data in datas:
        data_vector = get_avg_word2vec_vector(simple_preprocess(data), model_word2vec)
        score = np.dot(query_vector, data_vector)
        if np.isnan(score):
            score = 0
        scores.append(score)
    return np.array(scores)

## Algorithm BM25 (model BM25)**bold text**

1. Install lib for model BM25

In [25]:
!pip install rank_bm25

2. Import lib using BM25

In [27]:
import numpy as np

from rank_bm25 import BM25Okapi

3. Create model BM25

In [31]:
bm25_questions = BM25Okapi(tokenized_questions)

4. BM25 function

In [29]:
def get_bm25_scores(user_question, bm25_model):
    bm_25_score = bm25_model.get_scores(user_question)
    scores = (bm_25_score - np.min(bm_25_score)) / \
        (np.max(bm_25_score) - np.min(bm_25_score))
    # Thay thế giá trị nan bằng 0
    scores = np.nan_to_num(scores)
    return scores

In [32]:

def get_top_n_ranked_bm25(question, n=5):
    """
    Returns the top n ranked contexts based on BM25 score.

    Args:
        question (str): The question to rank contexts for.
        n (int): The number of top contexts to return. Default is 5.

    Returns:
        list: A list of tuples containing the context and its BM25 score.
    """
    bm25_scores = get_bm25_scores(question.split(), bm25_questions)
    top_n = np.argsort(bm25_scores)[::-1]
    data = []
    count = 0
    for index in top_n:
        if count == n:
            break
        tmp = {
            'score': f"{bm25_scores[index]:.4f}",
            'question': questions[index],
            'answer': answers[index]
        }
        if tmp in data:
            continue
        data.append(tmp)
        count += 1

    return data


6. Example top 5 question

In [36]:
ques = get_top_n_ranked_bm25("Luật lao động gồm bao nhiêu chương")
for q in ques:
  print(q)

{'score': '1.0000', 'question': 'chương 1 của bộ luật lao động việt nam 2019 gồm bao nhiêu điều', 'answer': {'text': 'chương 1 gồm từ điều 1 đến điều 6', 'start': 0}}
{'score': '0.7752', 'question': 'bộ luật lao động việt nam 2019 có bao nhiêu chương', 'answer': {'text': 'bộ luật lao động việt nam 2019 có 17 chương', 'start': 17}}
{'score': '0.7348', 'question': 'nội dung chính của chương vi bao gồm những gì', 'answer': {'text': 'nguyên tắc trả lương hình thức trả lương kỳ hạn trả lương tiền lương làm thêm giờ tiền lương làm việc vào ban đêm tiền lương nghỉ lễ tết và ngày nghỉ có hưởng lương tiền lương ngừng việc tiền lương cho người lao động bị tạm đình chỉ công việc mức lương tối thiểu và điều chỉnh mức lương tối thiểu thỏa thuận về tiền lương và quyền và nghĩa vụ của người sử dụng lao động và người lao động trong việc trả lương và nhận lương', 'start': 84}}
{'score': '0.7348', 'question': 'nội dung chính của chương iv bao gồm những gì', 'answer': {'text': 'quy định về học nghề đào t

## phoBERT

1. Install lib for model phoBERT

In [ ]:
!pip install datasets rouge_score
!pip install accelerate -U

2. Import lib for using phoBERT

In [59]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

3. Choose using model

In [39]:
model_path = './drive/MyDrive/models/phoBert_model'

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForQuestionAnswering.from_pretrained(model_path)

4. Function test combine model

In [40]:
def get_combined_scores(user_question, datas, model_word2vec, bm25_model):
    tokenized_query = user_question.split()
    word2vec_scores = get_word2vec_scores(
        tokenized_query, datas, model_word2vec)
    bm25_scores = get_bm25_scores(tokenized_query, bm25_model)
    word2vec_scores = (word2vec_scores - np.min(word2vec_scores)) / \
        (np.max(word2vec_scores) - np.min(word2vec_scores))
    combined_scores = word2vec_scores + bm25_scores
    # Thay thế giá trị nan bằng 0 trong combined_scores
    combined_scores = np.nan_to_num(combined_scores)
    return combined_scores

In [41]:
def get_combined_bm25_word2vec_scores(user_question):
    tokenized_query = user_question.split()
    word2vec_scores = get_word2vec_scores(
        tokenized_query, questions, word2vec_questions)
    bm25_scores = get_bm25_scores(tokenized_query, bm25_questions)
    word2vec_scores = (word2vec_scores - np.min(word2vec_scores)) / \
        (np.max(word2vec_scores) - np.min(word2vec_scores))
    combined_scores = word2vec_scores + bm25_scores
    # Thay thế giá trị nan bằng 0 trong combined_scores
    combined_scores = np.nan_to_num(combined_scores)
    if np.all(combined_scores == 0):
        return "Không có câu hỏi được tìm thấy", {'text': "Không có câu trả lời được tìm thấy"}

    best_match_idx = np.argmax(combined_scores)
    best_question = questions[best_match_idx]
    best_ans = answers[best_match_idx]
    print(best_question, best_ans)
    return best_question, best_ans

Test combine word2vec and BM25

In [42]:
print(get_combined_bm25_word2vec_scores("Bảo hiểm xã hội"))

ai phải tham gia bảo hiểm xã hội bắt buộc cho người lao động {'text': 'người sử dụng lao động phải tham gia bảo hiểm xã hội bắt buộc cho người lao động', 'start': 27}
('ai phải tham gia bảo hiểm xã hội bắt buộc cho người lao động', {'text': 'người sử dụng lao động phải tham gia bảo hiểm xã hội bắt buộc cho người lao động', 'start': 27})


In [43]:
def find_best_matching_question(user_question, questions, context_raws, question_raws):
    combined_scores = get_combined_scores(
        user_question, questions, word2vec_questions, bm25_questions)
    if np.all(combined_scores == 0):
        return "", "", "Không có câu hỏi được tìm thấy"
    best_match_idx = np.argmax(combined_scores)
    return question_raws[best_match_idx], context_raws[best_match_idx], answers[best_match_idx]

In [57]:
def get_predicted_answer(question):
    best_question, match_context, ans = find_best_matching_question(
        question, questions, context_raws, question_raws)

    inputs = tokenizer.encode_plus(
        best_question, match_context, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    outputs = model(**inputs)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    return answer

Test predict of phoBERT model

In [62]:
print(get_predicted_answer("bảo hiểm xã hội"))

Người sử dụng lao động phải tham gia bảo hiểm xã hội bắt buộc cho người lao động theo


In [66]:
def get_answer(question):
    bestQuestion, contextMatch, answerMatch = find_best_matching_question(
        question, questions, context_raws, question_raws)
    inputs = tokenizer(bestQuestion.lower(), contextMatch, return_tensors="pt",
                       max_length=128, padding="max_length", truncation="only_second")
    with torch.no_grad():
        outputs = model(**inputs)
    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()
    predict_answer_tokens = inputs.input_ids[0,
                                             answer_start_index: answer_end_index + 1]
    answer_result = tokenizer.decode(predict_answer_tokens)

    if len(answerMatch['text'].strip()) > len(answer_result.strip()):
        return answerMatch['text']
    elif (tokenizer.decode(predict_answer_tokens) == ""):
        return "Chưa thể tìm thấy câu trả lời!"
    else:
        return tokenizer.decode(predict_answer_tokens)

Test combine BM25 + word2vec + phoBERT

In [67]:
print(get_answer("Hợp đồng thử việc"))

Các bên có quyền thỏa thuận về việc chấm dứt hợp đồng thử việc. 2.
